In [45]:
#!pip install emoji
#!pip install spacy
import pandas as pd
import numpy as np
import emoji
import regex
import nltk
import re
import sklearn
import string
import spacy
from collections import Counter

In [46]:
df = pd.read_csv('data2021.csv')
print(len(df))
df = df.drop_duplicates(subset=['user_id']) # remove duplicates

392446


In [47]:
df.dropna(subset=['user_description'], inplace=True) # remove NaN values

In [48]:
df['user_description']

1         Wiskunde lover, honden freak. 🇮🇱🇹🇭🏁🇩🇪🇭🇺🇳🇱🇷🇸🇸🇽🇸...
2         IT ICT IoT || •AgriFoodTech •Biobased •Circula...
6         Leesbaar, kroniek van gebeurtenissen in Amsterdam
7         Mijn #TOTELTUIN ++ #Noseflutejob ++ Bach on th...
8         Columnist https://t.co/eX0BHLeWH2. Boosdrietig...
                                ...                        
392401    Je suis un 🤖 veillant sur la qualité de l'air ...
392414    Voor het meest uitgebreide zakelijke nieuws be...
392416    Uitgeverij Historische Verhalen publiceert kor...
392422    Dutch,Jewish,single,had a great friend for lif...
392442     Gitarist (klassiek) 🎸🎼🎵🎶 en 🖥️🗃️🛠️ 😎 IT-Goeroe 😎
Name: user_description, Length: 92363, dtype: object

In [49]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.EMOJI_DATA]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
df['emoji_free_tweets'] = df['user_description'].apply(call_emoji_free)

#Create a new column with url free tweets
df['url_free_tweets'] = df['emoji_free_tweets'].apply(url_free_text)

In [50]:
df['url_free_tweets'].str.rstrip()

1                             Wiskunde lover, honden freak.
2         IT ICT IoT || •AgriFoodTech •Biobased •Circula...
6         Leesbaar, kroniek van gebeurtenissen in Amsterdam
7         Mijn #TOTELTUIN ++ #Noseflutejob ++ Bach on th...
8         Columnist  Boosdrietig vanwege een tweet? Jamm...
                                ...                        
392401    Je suis un veillant sur la qualité de l'air qu...
392414    Voor het meest uitgebreide zakelijke nieuws be...
392416    Uitgeverij Historische Verhalen publiceert kor...
392422    Dutch,Jewish,single,had a great friend for lif...
392442                     Gitarist (klassiek) en IT-Goeroe
Name: url_free_tweets, Length: 92363, dtype: object

In [51]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

wordnet_lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('dutch')
def preprocessing(tweet):
    '''preprocess a tweet'''
    
    tweet = "".join([i for i in tweet if i not in string.punctuation]) # remove punctuation
    tweet = tweet.lower() # lowercase
    
    tweet = word_tokenize(tweet, language="dutch") # tokenize the tweet
    
    tweet = [i for i in tweet if i not in stopwords] # remove stopwords
    
    tweet = [wordnet_lemmatizer.lemmatize(word) for word in tweet] # lemmatize the tweets
    
    return tweet
    

In [58]:
df['preprocessed'] = df['url_free_tweets'].apply(preprocessing)
df = df[df.astype(str)['preprocessed'] != '[]']

In [59]:
df['preprocessed']

1                          [wiskunde, lover, honden, freak]
2         [it, ict, iot, •agrifoodtech, •biobased, •circ...
6            [leesbaar, kroniek, gebeurtenissen, amsterdam]
7         [toteltuin, noseflutejob, bach, on, the, sheep...
8         [columnist, boosdrietig, vanwege, tweet, jamme...
                                ...                        
392401    [suis, un, veillant, sur, la, qualité, lair, q...
392414    [meest, uitgebreide, zakelijke, nieuws, bezoek...
392416    [uitgeverij, historische, verhalen, publiceert...
392422    [dutchjewishsinglehad, a, great, friend, for, ...
392442                       [gitarist, klassiek, itgoeroe]
Name: preprocessed, Length: 91003, dtype: object